In [1]:
from google.colab import drive
drive.mount('/content/drive/')

#change the directory manually if you are not marco or allan
who_is_accesing = input("say your name ")
if who_is_accesing == "marco":
  %cd "/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3"
  directory  = "/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3"
elif who_is_accesing == "allan":
  %cd "/content/drive/MyDrive/Summer 2022/task_3"
  directory = "/content/drive/MyDrive/Summer 2022/task_3"
elif who_is_accesing == "mldl":
  %cd "/content/drive/MyDrive/task_3"
  directory = "/content/drive/MyDrive/task_3"


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
say your name marco
/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3


In [2]:
import argparse
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
#Models
from model.build_BiSeNet import BiSeNet
from model.discriminator import Discriminator

#losses
from loss import CrossEntropyLoss
from torch.nn import BCEWithLogitsLoss
from loss import DiceLoss

#datasets
from dataset.cityscapes_dataset import CityScapeDataSet
from dataset.gta5_dataset import GTA5DataSet
from dataset.cityscapes_dataset_val import CityScapeDataSet_val

import torch
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
from utils import poly_lr_scheduler
from utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu
from torch.autograd import Variable

import torch.cuda.amp as amp
import argparse

In [3]:
def validation_segmentation(args, modelSEG, dataloader):
  print('start validation')
  with torch.no_grad():
    modelSEG.eval()
    precision_record = []
    hist = np.zeros((args.num_classes, args.num_classes))

    for i, (image, label) in enumerate(dataloader):
      label = label.type(torch.LongTensor)
      label = label.long().cuda()
      image = image.cuda()

      # get RGB predict image
      predict = modelSEG(image).squeeze()
      predict = reverse_one_hot(predict)
      predict = np.array(predict.item)

      # get RGB label image
      label = label.squeeze()
      label = np.array(label.cpu())

      # compute per pixel accuracy
      precision = compute_global_accuracy(predict, label)
      hist += fast_hist(label.flatten(), predict.flatten(), args.num_classes)
      
      precision_record.append(precision)

    precision = np.mean(precision_record)
    miou_list = per_class_iu(hist)
    miou = np.mean(miou_list)
    print('precision per pixel for test: %.3f' % precision)
    print('mIoU for validation: %.3f' % miou)
    print(f'mIoU per class: {miou_list}')

  return precision, miou  


In [4]:
def train_model(args, modelSEG, modelADV, optimizerSEG, optimizerADV, dataloader_target, dataloader_source, model_seg_dictionary = None, model_adv_dictionary = None):
  
  #initialize the dictionaries for the checkpoints
  if model_seg_dictionary != None and model_adv_dictionary != None:
    print("Using checkpoints")
    modelSEG.load_state_dict(model_seg_dictionary['model_state_dict'])
    modelADV.load_state_dict(model_adv_dictionary['model_state_dict'])

    optimizerSEG.load_state_dict(model_seg_dictionary['optimizer_state_dict'])
    optimizerADV.load_state_dict(model_adv_dictionary['optimizer_state_dict'])

    initial_epoch = model_seg_dictionary["epoch"]
    checkpoint = True

  else:
    initial_epoch = 0

  #initializing scaler object
  scaler = amp.GradScaler()

  #set the model to train mode
  modelSEG.train()
  modelADV.train()


  #Setting the losses functions
  loss_seg_source = torch.nn.CrossEntropyLoss(ignore_index=255) #loss to train the segmentation network with the source dataset
  loss_adv_target_source = torch.nn.BCEWithLogitsLoss() #loss to train the segmenation network with the target dataset via the adversarial network

  loss_D = torch.nn.BCEWithLogitsLoss() #loss to train only the adversarial network

  # labels for adversarial training
  source_label = 0
  target_label = 1

  #Defining the optmizers
  optmizer_segmentation = optimizerSEG
  optmizer_adversarial = optimizerADV

  epoch = initial_epoch
  while epoch < args.num_epochs:
    print("Epoch = ",epoch)

    #print("Epoch number", epoch)  

    #Save the loss for each epoch
    loss_value_seg_source = []
    loss_value_seg_target = []
    
    loss_value_adv_source = []
    loss_value_adv_target = []
    pres = 0
    prev = 0
    #Defining the iterator object for the dataloaders
    dataloader_source_iter = enumerate(dataloader_source)
    dataloader_target_iter = enumerate(dataloader_target)  

    for i_iter in tqdm(range(args.number_steps)):

      #zero de gradient at the optmizer for each iteration
      
      optmizer_segmentation.zero_grad()
      lr_seg = poly_lr_scheduler(optimizerSEG, args.learning_rate, iter=i_iter, max_iter=args.number_steps, lr_decay_iter = 25)
      print("lr_seg", lr_seg)

      optmizer_adversarial.zero_grad()
      lr_adv = poly_lr_scheduler(optimizerADV, args.learning_rate_adversarial, iter=i_iter, max_iter=args.number_steps, lr_decay_iter = 25)
      print("lr_adv",lr_adv)




      ###Train the segmentation net 
      # don't accumulate grads in D while training the segmentation network
      for param in modelADV.parameters():
        param.requires_grad = False
      
      ##Train using the source dataset

      try:
        _, batch_source = dataloader_source_iter.__next__()
      except:
        dataloader_source_iter = enumerate(dataloader_source)
        _, batch_source = dataloader_source_iter.__next__()

      images_source, labels_source = batch_source
      images_source = Variable(images_source).cuda()
      labels_source = labels_source.long().cuda()


      #Computing the losses from the output of the seg net
      with amp.autocast(): 
        output_source_seg_seg, output_sup1_source_seg, output_sup2_source_seg = modelSEG(images_source)
        
        loss1_source_seg = loss_seg_source(output_source_seg_seg, labels_source)
        loss2_source_seg = loss_seg_source(output_sup1_source_seg, labels_source)
        loss3_source_seg = loss_seg_source(output_sup2_source_seg, labels_source)
        loss_source_seg = loss1_source_seg + loss2_source_seg + loss3_source_seg
        print("\n \n \n the loss is",loss_source_seg, "\n \n \n")
        
      #Backpropagating the loss
      
      scaler.scale(loss_source_seg).backward(retain_graph=True)

      #debug
      print(prev - (list(modelSEG.parameters())[-2]))
      prev = (list(modelSEG.parameters())[-2])
      #debug

      loss_value_seg_source.append(loss_source_seg.item())

      ##Train using the target dataset
      try:
        _, batch_target = dataloader_target_iter.__next__()
      except:
        dataloader_target_iter = enumerate(dataloader_target)
        _, batch_target = dataloader_target_iter.__next__()

      images_target = batch_target
      images_target = Variable(images_target).cuda()
      
      #Computing the loss of the seg net from target

      with amp.autocast(): 
        output_target_seg_seg, _, _ = modelSEG(images_target)
        output_target_adv_seg = modelADV(F.softmax(output_target_seg_seg))

        loss_buffer_target_seg = loss_adv_target_source(output_target_adv_seg, Variable(torch.FloatTensor(output_target_adv_seg.data.size()).fill_(source_label)).cuda())
        loss_target_seg = args.lambda_adv_target * loss_buffer_target_seg


      #Backpropagating the loss
      scaler.scale(loss_target_seg).backward(retain_graph=True)
      loss_value_seg_target.append(loss_target_seg.item())
    
      
      ###Train the adversarial net
      #Start to accumulate grads in D while training the adversarial network
      for param in modelADV.parameters():
        param.requires_grad = True

      ##Train adversarial network with source dataset
      #detatching the output_source from the segnet
      output_source_seg_seg.detach()

      #computing the loss of the adv net from source
      with amp.autocast():
        output_source_adv = modelADV(F.softmax(output_source_seg_seg))
        loss_D_source = loss_D(output_source_adv,Variable(torch.FloatTensor(output_source_adv.data.size()).fill_(source_label)).cuda())
        loss_D_source = loss_D_source/2

      #Backpropagating the loss
      scaler.scale(loss_D_source).backward(retain_graph=True)
      loss_value_adv_source.append(loss_D_source.item())



      ##Train adversarial network with target dataset
      #detatching the output_source from the segnet
      output_target_seg_seg.detach()

      #computing the loss of the adv net from target
      with amp.autocast():
        output_target_adv = modelADV(F.softmax(output_target_seg_seg))
        loss_D_target = loss_D(output_target_adv,Variable(torch.FloatTensor(output_target_adv.data.size()).fill_(target_label)).cuda())
        loss_D_target = loss_D_target/2

      #Backpropagating the loss
  
      scaler.scale(loss_D_target).backward(retain_graph=True)

      loss_value_adv_target.append(loss_D_target.item())


      #Make the itearation with the optmizer
      scaler.step(optmizer_adversarial)
      scaler.step(optmizer_segmentation)
      scaler.update()

    print("loss Seg = ", np.mean(loss_value_seg_source) + np.mean(loss_value_seg_target))
    print("loss Adv = ", np.mean(loss_value_adv_source) + np.mean(loss_value_adv_target))
    epoch += 1

    #Saving Checkpoint
    if (epoch % args.checkpoint_step) == 0:
      

      EPOCH = epoch
      PATH_SEG = args.checkpoint_directory + "/SEG/modelSEG.pt"
      PATH_ADV = args.checkpoint_directory + "/ADV/modelADV.pt"
      LOSS_MEAN_SEG = np.mean(loss_value_seg_source) + np.mean(loss_value_seg_target)
      LOSS_MEAN_ADV = np.mean(loss_value_adv_source) + np.mean(loss_value_adv_target)

      torch.save({
          'epoch': EPOCH,
          'model_state_dict': modelSEG.state_dict(),
          'optimizer_state_dict': optmizer_segmentation.state_dict(),
          'loss': LOSS_MEAN_SEG,
          }, PATH_SEG)
      
      torch.save({
          'epoch': EPOCH,
          'model_state_dict': modelADV.state_dict(),
          'optimizer_state_dict': optmizer_adversarial.state_dict(),
          'loss': LOSS_MEAN_ADV,
          }, PATH_ADV)





In [5]:
def main(params):
  parser = argparse.ArgumentParser()

  parser.add_argument('--cityscapes_dir', type=str,
                      default=(directory + "/dataset/Cityscapes"), 
                      help='Directory of the cityscapes dataset')
  parser.add_argument('--gta5_dir', type=str,
                      default=(directory+"/dataset/GTA5"), 
                      help='Directory of the GTA5 dataset')
  
  parser.add_argument('--image_dimensions', type=tuple,
                        default=(512,1024), 
                        help='Dimensions of images')

  parser.add_argument('--num_epochs', type=int, default=26  , help='Number of epochs to train for')
  parser.add_argument('--checkpoint_directory', type=str, default = (directory + "/checkpoints") , help='Directory of the checkpoints folder')
  parser.add_argument('--checkpoint_step', type=int, default=1, help='How often to save checkpoints (epochs)')
  parser.add_argument('--number_steps', type=int, default=1500, help='')

  parser.add_argument('--batch_size', type=int, default=4, help='Number of images in each batch')
  parser.add_argument('--context_path_model', type=str, default="resnet101",
                      help='The context path model you are using, resnet18, resnet101.')
  parser.add_argument('--learning_rate', type=float, default=0.00025, help='learning rate used for train')
  parser.add_argument('--learning_rate_adversarial', type=float, default=0.00025, help='learning rate used for train de adersarial network')
  parser.add_argument("--lambda_adv_target", type=float, default=0.001, help="lambda_adv for adversarial training.")
  parser.add_argument('--data', type=str, default='', help='path of training data')
  parser.add_argument('--num_workers', type=int, default=4, help='num of workers')
  parser.add_argument('--num_classes', type=int, default=19, help='num of object classes (with void)')
  parser.add_argument('--cuda', type=str, default='0', help='GPU ids used for training')
  parser.add_argument('--use_gpu', type=bool, default=True, help='whether to user gpu for training')
  parser.add_argument('--pretrained_model_path', type=str, default=None, help='path to pretrained model')
  parser.add_argument('--save_model_path', type=str, default=None, help='path to save model')


  args = parser.parse_args(params)

  #loading data
  dataset_target = CityScapeDataSet(args.cityscapes_dir, os.path.join(args.cityscapes_dir, "images"), "train.txt", target_dimensions = args.image_dimensions)
  dataset_source = GTA5DataSet(args.gta5_dir, os.path.join(args.gta5_dir, "images"), os.path.join(args.gta5_dir, "labels"), "train.txt", image_dimensions = args.image_dimensions)
  dataset_val = CityScapeDataSet_val(args.cityscapes_dir, os.path.join(args.cityscapes_dir, "images"), os.path.join(args.cityscapes_dir, "labels"), "val.txt", image_dimensions = args.image_dimensions)
  print(len(dataset_target))

  dataloader_target = DataLoader(dataset_target, batch_size = args.batch_size, num_workers = args.num_workers, shuffle = True, drop_last= True, persistent_workers= True)
  dataloader_source = DataLoader(dataset_source, batch_size = args.batch_size, num_workers = args.num_workers, shuffle = True, drop_last= True, persistent_workers= True)
  dataloader_val = DataLoader(dataset_val, batch_size = 1, num_workers = args.num_workers, shuffle = True, drop_last= False)
  

  #initialize models
  os.environ['CUDA_VISIBLE_DEVICES'] = args.cuda

  modelSEG = BiSeNet(args.num_classes, "resnet18")
  modelADV = Discriminator(args.num_classes)
  
  if torch.cuda.is_available() and args.use_gpu:
      modelSEG = torch.nn.DataParallel(modelSEG).cuda()
      modelADV = torch.nn.DataParallel(modelADV).cuda()
  
  #Here we have to to define the optmizer for the SEG
  optmizer_segmentation = torch.optim.SGD(modelSEG.parameters(), args.learning_rate, momentum=0.9, weight_decay=1e-4)
  optmizer_adversarial = torch.optim.Adam(modelADV.parameters(), lr=args.learning_rate_adversarial, betas=(0.9, 0.99))


  #Import checkpoints
  use_checkpoints = False
  if use_checkpoints:
    checkpoint_seg = torch.load(args.checkpoint_directory + "/SEG/modelSEG.pt")
    checkpoint_adv = torch.load(args.checkpoint_directory + "/ADV/modelADV.pt")
  else:
    checkpoint_seg = None
    checkpoint_adv = None

  #Doing the calculation for the number of iterations
  size_iterations = len(dataloader_source.dataset)/args.batch_size
  params_update = ['--number_steps', str(int(size_iterations))]
  args = parser.parse_args(params_update)


  train_model(args, modelSEG, modelADV, optmizer_segmentation, optmizer_adversarial, dataloader_target, dataloader_source,checkpoint_seg, checkpoint_adv)

  precision, MIOU = validation_segmentation(args, modelSEG, dataloader_val)

  

  #Here we can define the pretrained model


In [6]:
main([])

425
Epoch =  0


  0%|          | 0/125 [00:00<?, ?it/s]

lr_seg 0.0025
lr_adv 0.0025

 
 
 the loss is tensor(9.8619, device='cuda:0', grad_fn=<AddBackward0>) 
 
 

tensor([[[[-4.0974e-02]],

         [[ 4.6201e-01]],

         [[ 3.0769e-01]],

         [[ 4.5260e-01]],

         [[-7.0949e-02]],

         [[-7.1236e-02]],

         [[ 6.2122e-01]],

         [[ 1.8355e-01]],

         [[-2.2681e-01]],

         [[ 1.6413e-01]],

         [[-6.1205e-01]],

         [[ 2.6208e-01]],

         [[-2.6395e-01]],

         [[-1.9534e-01]],

         [[ 1.6598e-01]],

         [[ 2.2249e-02]],

         [[-7.0109e-01]],

         [[ 3.5874e-01]],

         [[-1.1485e-02]]],


        [[[-4.9865e-01]],

         [[-1.7113e-02]],

         [[ 4.1825e-01]],

         [[ 1.5570e-01]],

         [[-1.6190e-01]],

         [[-5.7873e-01]],

         [[-4.4133e-01]],

         [[ 3.0005e-01]],

         [[-4.4411e-01]],

         [[-9.2305e-02]],

         [[ 4.7571e-03]],

         [[-4.2812e-01]],

         [[-7.4518e-02]],

         [[-4.3130e-01]],


  1%|          | 1/125 [00:06<12:35,  6.09s/it]

lr_seg 0.0024819927787908536
lr_adv 0.0024819927787908536

 
 
 the loss is tensor(9.3752, device='cuda:0', grad_fn=<AddBackward0>) 
 
 

tensor([[[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0

...

lr_seg 0.0022283038652225
lr_adv 0.0022283038652225

 
 
 the loss is tensor(6.4360, device='cuda:0', grad_fn=<AddBackward0>) 
 
 

tensor([[[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]],


        [[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],



 13%|█▎        | 16/125 [00:22<01:59,  1.10s/it]

lr_seg 0.002210063973306727
lr_adv 0.002210063973306727


 13%|█▎        | 16/125 [00:26<03:02,  1.67s/it]


KeyboardInterrupt: ignored